# Debugging

**Teaching:** 30 min  
**Exercises:** 20 min

## Learning Objectives

- Debug code containing an error systematically.
- Identify ways of making code less error-prone and more easily tested.
- Apply systematic debugging principles to find and fix bugs.
- Use debugging tools and techniques effectively.

## Questions

- How can I debug my program?
- What tools and techniques help find bugs quickly?
- How can I make my code easier to debug?

---

Once testing has uncovered problems, the next step is to fix them. Many novices do this by making more-or-less random changes to their code until it seems to produce the right answer, but that's very inefficient (and the result is usually only correct for the one case they're testing).

The more experienced a programmer is, the more systematically they debug, and most follow some variation on the rules we'll explore in this lesson.

## Setup

Let's start by importing the libraries we'll need and loading our inflammation data:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import traceback
from pdb import set_trace as breakpoint  # Python debugger

In [ ]:
# Load our inflammation data
data = np.loadtxt('../data/inflammation-01.csv', delimiter=',')
print(f"Data shape: {data.shape}")
print(f"Data type: {data.dtype}")

## Debugging Principles

Before we dive into specific techniques, let's establish the fundamental principles of systematic debugging:

1. **Know what it's supposed to do**
2. **Make it fail every time**
3. **Make it fail fast**
4. **Change one thing at a time, for a reason**
5. **Keep track of what you've done**
6. **Be humble**

Let's explore each of these with practical examples.

## 1. Know What It's Supposed to Do

The first step in debugging is to know what your code is supposed to do. "My program doesn't work" isn't good enough – you need to be able to tell correct output from incorrect.

For scientific software, we can:
- Test with simplified data
- Test simplified cases
- Compare to trusted results (oracles)
- Check conservation laws
- Visualize results

In [ ]:
# Example: A function with a bug - let's see if we can spot it
def calculate_average_inflammation(data):
    """Calculate average inflammation for each day."""
    # This function has a subtle bug - can you spot it?
    daily_averages = []
    for day in range(data.shape[1]):
        day_data = data[:, day]
        average = sum(day_data) / len(data)  # BUG: should be len(day_data)
        daily_averages.append(average)
    return daily_averages

# Test with our data
averages = calculate_average_inflammation(data)
print(f"First 10 daily averages: {averages[:10]}")
print(f"Number of averages: {len(averages)}")

In [ ]:
# Let's test with simplified data where we know the answer
simple_data = np.array([[1, 2, 3],
                       [4, 5, 6]])
print(f"Simple data:\n{simple_data}")
print(f"Expected averages: [2.5, 3.5, 4.5]")

simple_averages = calculate_average_inflammation(simple_data)
print(f"Calculated averages: {simple_averages}")
print(f"Something's wrong! The averages should be [2.5, 3.5, 4.5]")

In [ ]:
# Let's compare with NumPy's built-in function (our oracle)
numpy_averages = np.mean(simple_data, axis=0)
print(f"NumPy averages: {numpy_averages}")
print(f"Our averages: {simple_averages}")
print(f"Match: {np.allclose(numpy_averages, simple_averages)}")

In [ ]:
# Fixed version
def calculate_average_inflammation_fixed(data):
    """Calculate average inflammation for each day (corrected version)."""
    daily_averages = []
    for day in range(data.shape[1]):
        day_data = data[:, day]
        average = sum(day_data) / len(day_data)  # FIXED: use len(day_data)
        daily_averages.append(average)
    return daily_averages

# Test the fixed version
fixed_averages = calculate_average_inflammation_fixed(simple_data)
print(f"Fixed averages: {fixed_averages}")
print(f"NumPy averages: {numpy_averages}")
print(f"Match: {np.allclose(numpy_averages, fixed_averages)}")

## 2. Make It Fail Every Time

We can only debug something when it fails, so the second step is always to find a test case that makes it fail every time. Intermittent problems are much harder to debug.

In [ ]:
# Example: A function that sometimes fails
def unreliable_division(numbers):
    """Divide each number by the next one in the list."""
    results = []
    for i in range(len(numbers)):
        # This will fail when we reach the last element
        try:
            result = numbers[i] / numbers[i + 1]
            results.append(result)
        except IndexError:
            print(f"Index error at position {i}")
            break
        except ZeroDivisionError:
            print(f"Division by zero at position {i}")
            results.append(float('inf'))
    return results

# Test cases that make it fail consistently
test_cases = [
    [1, 2, 3, 4],      # Normal case
    [1, 0, 3],         # Division by zero
    [5],               # Single element (index error)
    [],                # Empty list
]

for i, test in enumerate(test_cases):
    print(f"\nTest {i+1}: {test}")
    result = unreliable_division(test)
    print(f"Result: {result}")

## 3. Make It Fail Fast

If it takes a long time for a bug to surface, debugging becomes inefficient. We want to localize the failure to the smallest possible region of code.

In [ ]:
# Example: Using print statements for quick debugging
def analyze_patient_data(patient_data):
    """Analyze patient inflammation data with debugging prints."""
    print(f"DEBUG: Input data shape: {patient_data.shape}")
    
    # Check for obvious problems early
    if patient_data.size == 0:
        print("DEBUG: Empty data detected!")
        return None
    
    if np.any(patient_data < 0):
        print(f"DEBUG: Negative values found: {np.sum(patient_data < 0)} values")
    
    # Calculate statistics
    mean_inflammation = np.mean(patient_data)
    max_inflammation = np.max(patient_data)
    min_inflammation = np.min(patient_data)
    
    print(f"DEBUG: Stats calculated - mean: {mean_inflammation:.2f}, range: {min_inflammation}-{max_inflammation}")
    
    # Check for suspicious results
    if mean_inflammation > 20:
        print("DEBUG: Warning - very high average inflammation!")
    
    return {
        'mean': mean_inflammation,
        'max': max_inflammation,
        'min': min_inflammation
    }

# Test with our data
result = analyze_patient_data(data)
print(f"\nFinal result: {result}")

## 4. Change One Thing at a Time, For a Reason

Good programmers change one thing at a time, for a reason. They are either trying to gather more information or test a specific fix.

In [ ]:
# Example: Systematic debugging of a BMI calculation
# This code has multiple bugs - let's fix them one at a time

patients = [[70, 1.8], [80, 1.9], [150, 1.7]]

def calculate_bmi_buggy(weight, height):
    """Calculate BMI with bugs."""
    return weight / (height ** 2)

print("=== BUGGY VERSION ===")
for patient in patients:
    weight, height = patients[0]  # BUG 1: Always using first patient
    bmi = calculate_bmi_buggy(height, weight)  # BUG 2: Arguments swapped
    print(f"Patient's BMI is: {bmi:.6f}")

print("\n=== FIX 1: Use correct patient data ===")
for patient in patients:
    weight, height = patient  # FIXED: Use current patient
    bmi = calculate_bmi_buggy(height, weight)  # Still has bug 2
    print(f"Patient (w:{weight}, h:{height}) BMI: {bmi:.6f}")

print("\n=== FIX 2: Correct argument order ===")
for patient in patients:
    weight, height = patient
    bmi = calculate_bmi_buggy(weight, height)  # FIXED: Correct order
    print(f"Patient (w:{weight}, h:{height}) BMI: {bmi:.2f}")

## 5. Keep Track of What You've Done

Good debugging requires keeping track of what you've tried and what worked. This helps avoid repeating unsuccessful approaches and provides valuable information when asking for help.

In [ ]:
# Example: Debugging log for tracking attempts
debugging_log = []

def log_debug_attempt(attempt, description, result):
    """Log a debugging attempt."""
    debugging_log.append({
        'attempt': attempt,
        'description': description,
        'result': result,
        'timestamp': 'simulated'
    })

# Simulate debugging attempts
log_debug_attempt(1, "Added print statements to track variable values", "Found that loop variable not updating")
log_debug_attempt(2, "Fixed loop variable issue", "Still getting wrong results")
log_debug_attempt(3, "Checked function argument order", "Arguments were swapped - fixed")
log_debug_attempt(4, "Re-ran all tests", "All tests now pass")

print("=== DEBUGGING LOG ===")
for entry in debugging_log:
    print(f"Attempt {entry['attempt']}: {entry['description']}")
    print(f"  Result: {entry['result']}\n")

## Advanced Debugging Techniques

### Using the Python Debugger (pdb)

Python's built-in debugger `pdb` allows you to step through code line by line, inspect variables, and understand program flow.

In [ ]:
def complex_calculation(data):
    """A function that might benefit from stepping through with debugger."""
    result = 0
    for i, value in enumerate(data):
        # Uncomment the next line to set a breakpoint
        # breakpoint()  # This would pause execution here
        
        if i % 2 == 0:
            result += value * 2
        else:
            result += value / 2
        
        # Debug print instead of breakpoint for demonstration
        print(f"Step {i}: value={value}, result={result}")
    
    return result

# Test the function
test_data = [1, 2, 3, 4, 5]
result = complex_calculation(test_data)
print(f"\nFinal result: {result}")

### Exception Handling and Traceback Analysis

In [ ]:
def analyze_with_error_handling(data_files):
    """Analyze multiple data files with comprehensive error handling."""
    results = []
    errors = []
    
    for filename in data_files:
        try:
            # Simulate loading data
            if 'missing' in filename:
                raise FileNotFoundError(f"File {filename} not found")
            elif 'corrupt' in filename:
                raise ValueError(f"Invalid data format in {filename}")
            else:
                # Simulate successful processing
                results.append(f"Processed {filename}")
                
        except FileNotFoundError as e:
            error_info = {
                'filename': filename,
                'error_type': 'FileNotFoundError',
                'message': str(e),
                'traceback': traceback.format_exc()
            }
            errors.append(error_info)
            print(f"ERROR: {e}")
            
        except ValueError as e:
            error_info = {
                'filename': filename,
                'error_type': 'ValueError',
                'message': str(e),
                'traceback': traceback.format_exc()
            }
            errors.append(error_info)
            print(f"ERROR: {e}")
            
        except Exception as e:
            error_info = {
                'filename': filename,
                'error_type': 'UnexpectedError',
                'message': str(e),
                'traceback': traceback.format_exc()
            }
            errors.append(error_info)
            print(f"UNEXPECTED ERROR: {e}")
    
    return results, errors

# Test with various scenarios
test_files = ['good_file.csv', 'missing_file.csv', 'corrupt_data.csv', 'another_good.csv']
results, errors = analyze_with_error_handling(test_files)

print(f"\nSuccessful results: {len(results)}")
print(f"Errors encountered: {len(errors)}")
for error in errors:
    print(f"  - {error['error_type']}: {error['message']}")

## Debugging Inflammation Data Analysis

Let's apply our debugging principles to a more complex inflammation data analysis function:

In [ ]:
def analyze_inflammation_trends(data, debug=False):
    """Analyze inflammation trends with debugging support."""
    if debug:
        print(f"DEBUG: Starting analysis with data shape {data.shape}")
    
    # Validate input
    assert data.ndim == 2, f"Expected 2D data, got {data.ndim}D"
    assert data.size > 0, "Data cannot be empty"
    
    patients, days = data.shape
    if debug:
        print(f"DEBUG: Analyzing {patients} patients over {days} days")
    
    # Calculate daily averages
    daily_averages = np.mean(data, axis=0)
    if debug:
        print(f"DEBUG: Daily averages range: {np.min(daily_averages):.2f} to {np.max(daily_averages):.2f}")
    
    # Find peak inflammation day
    peak_day = np.argmax(daily_averages)
    peak_value = daily_averages[peak_day]
    if debug:
        print(f"DEBUG: Peak inflammation on day {peak_day} with value {peak_value:.2f}")
    
    # Calculate trends
    first_half = daily_averages[:days//2]
    second_half = daily_averages[days//2:]
    
    first_half_avg = np.mean(first_half)
    second_half_avg = np.mean(second_half)
    
    trend = "increasing" if second_half_avg > first_half_avg else "decreasing"
    
    if debug:
        print(f"DEBUG: First half avg: {first_half_avg:.2f}, Second half avg: {second_half_avg:.2f}")
        print(f"DEBUG: Overall trend: {trend}")
    
    # Check for suspicious patterns
    warnings = []
    if np.all(daily_averages == daily_averages[0]):
        warnings.append("All daily averages are identical")
    
    if peak_value > 15:
        warnings.append(f"Very high peak inflammation: {peak_value:.2f}")
    
    if debug and warnings:
        print(f"DEBUG: Warnings: {warnings}")
    
    return {
        'daily_averages': daily_averages,
        'peak_day': peak_day,
        'peak_value': peak_value,
        'trend': trend,
        'first_half_avg': first_half_avg,
        'second_half_avg': second_half_avg,
        'warnings': warnings
    }

# Test with debugging enabled
result = analyze_inflammation_trends(data, debug=True)

print("\n=== ANALYSIS RESULTS ===")
print(f"Peak inflammation: Day {result['peak_day']} (value: {result['peak_value']:.2f})")
print(f"Overall trend: {result['trend']}")
print(f"Warnings: {result['warnings'] if result['warnings'] else 'None'}")

## Visualization for Debugging

Sometimes the best way to debug is to visualize what's happening:

In [ ]:
def debug_with_plots(data):
    """Use plots to debug data analysis."""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot 1: Raw data heatmap
    im1 = axes[0,0].imshow(data, cmap='hot', aspect='auto')
    axes[0,0].set_title('Raw Inflammation Data')
    axes[0,0].set_xlabel('Day')
    axes[0,0].set_ylabel('Patient')
    plt.colorbar(im1, ax=axes[0,0])
    
    # Plot 2: Daily averages
    daily_avg = np.mean(data, axis=0)
    axes[0,1].plot(daily_avg, 'b-', linewidth=2)
    axes[0,1].set_title('Daily Average Inflammation')
    axes[0,1].set_xlabel('Day')
    axes[0,1].set_ylabel('Average Inflammation')
    axes[0,1].grid(True, alpha=0.3)
    
    # Plot 3: Patient averages
    patient_avg = np.mean(data, axis=1)
    axes[1,0].hist(patient_avg, bins=20, alpha=0.7, edgecolor='black')
    axes[1,0].set_title('Distribution of Patient Average Inflammation')
    axes[1,0].set_xlabel('Average Inflammation')
    axes[1,0].set_ylabel('Number of Patients')
    
    # Plot 4: Data quality indicators
    max_daily = np.max(data, axis=0)
    min_daily = np.min(data, axis=0)
    axes[1,1].fill_between(range(len(daily_avg)), min_daily, max_daily, alpha=0.3, label='Range')
    axes[1,1].plot(daily_avg, 'r-', linewidth=2, label='Average')
    axes[1,1].set_title('Daily Inflammation Range')
    axes[1,1].set_xlabel('Day')
    axes[1,1].set_ylabel('Inflammation')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print some diagnostic information
    print("=== DATA QUALITY DIAGNOSTICS ===")
    print(f"Data shape: {data.shape}")
    print(f"Data range: {np.min(data):.2f} to {np.max(data):.2f}")
    print(f"Missing values: {np.sum(np.isnan(data))}")
    print(f"Negative values: {np.sum(data < 0)}")
    print(f"Zero values: {np.sum(data == 0)}")
    print(f"Suspiciously high values (>15): {np.sum(data > 15)}")

# Create debugging plots
debug_with_plots(data)

## Exercise: Debug the BMI Calculator

Here's a BMI calculator with several bugs. Use the debugging principles we've learned to find and fix them:

In [ ]:
# Exercise: Debug this BMI calculator
patients = [[70, 1.8], [80, 1.9], [150, 1.7]]

def calculate_bmi(weight, height):
    return weight / (height ** 2)

print("=== BUGGY BMI CALCULATOR ===")
for patient in patients:
    weight, height = patients[0]  # Bug 1: What's wrong here?
    bmi = calculate_bmi(height, weight)  # Bug 2: What's wrong here?
    print("Patient's BMI is:", bmi)

# Expected BMIs should be approximately:
# Patient 1: 70 / (1.8^2) = 21.60
# Patient 2: 80 / (1.9^2) = 22.16  
# Patient 3: 150 / (1.7^2) = 51.90

### Your debugging task:
1. Identify what's wrong with the current output
2. Add debugging prints to understand the problem
3. Fix one bug at a time
4. Verify each fix works before moving to the next
5. Test with known good values

In [ ]:
# Your solution here - use the cell below to debug step by step
print("=== DEBUGGING BMI CALCULATOR ===")

# Step 1: Add debugging prints
for i, patient in enumerate(patients):
    print(f"\nProcessing patient {i+1}: {patient}")
    weight, height = patients[0]  # Still buggy
    print(f"  Using weight: {weight}, height: {height}")
    bmi = calculate_bmi(height, weight)  # Still buggy
    print(f"  Calculated BMI: {bmi:.6f}")

In [ ]:
# Step 2: Fix bug 1 (wrong patient data)
print("=== FIXING BUG 1: Using correct patient data ===")
for i, patient in enumerate(patients):
    print(f"\nProcessing patient {i+1}: {patient}")
    weight, height = patient  # FIXED: use current patient
    print(f"  Using weight: {weight}, height: {height}")
    bmi = calculate_bmi(height, weight)  # Still has bug 2
    print(f"  Calculated BMI: {bmi:.6f}")

In [ ]:
# Step 3: Fix bug 2 (swapped arguments)
print("=== FIXING BUG 2: Correct argument order ===")
for i, patient in enumerate(patients):
    print(f"\nProcessing patient {i+1}: {patient}")
    weight, height = patient
    print(f"  Using weight: {weight}, height: {height}")
    bmi = calculate_bmi(weight, height)  # FIXED: correct order
    print(f"  Calculated BMI: {bmi:.2f}")
    
    # Verify the calculation manually
    manual_bmi = weight / (height ** 2)
    print(f"  Manual verification: {weight} / ({height}^2) = {manual_bmi:.2f}")
    print(f"  Match: {abs(bmi - manual_bmi) < 0.001}")

## Creating a Robust Debugging Framework

Let's create a reusable debugging framework for our inflammation analysis:

In [ ]:
class DebugLogger:
    """A simple debugging logger class."""
    
    def __init__(self, debug_enabled=True):
        self.debug_enabled = debug_enabled
        self.logs = []
    
    def log(self, message, level="INFO"):
        """Log a debugging message."""
        if self.debug_enabled:
            log_entry = f"[{level}] {message}"
            self.logs.append(log_entry)
            print(log_entry)
    
    def assert_condition(self, condition, message):
        """Assert a condition and log the result."""
        if condition:
            self.log(f"ASSERTION PASSED: {message}", "PASS")
        else:
            self.log(f"ASSERTION FAILED: {message}", "FAIL")
            raise AssertionError(message)
    
    def get_logs(self):
        """Return all logged messages."""
        return self.logs

def robust_inflammation_analysis(data, logger=None):
    """Perform inflammation analysis with comprehensive debugging."""
    if logger is None:
        logger = DebugLogger(debug_enabled=False)
    
    logger.log(f"Starting analysis with data shape {data.shape}")
    
    # Input validation with logging
    logger.assert_condition(isinstance(data, np.ndarray), "Input must be numpy array")
    logger.assert_condition(data.ndim == 2, "Data must be 2-dimensional")
    logger.assert_condition(data.size > 0, "Data cannot be empty")
    logger.assert_condition(not np.any(np.isnan(data)), "Data contains NaN values")
    
    # Data quality checks
    neg_count = np.sum(data < 0)
    if neg_count > 0:
        logger.log(f"WARNING: Found {neg_count} negative values", "WARN")
    
    high_count = np.sum(data > 20)
    if high_count > 0:
        logger.log(f"WARNING: Found {high_count} unusually high values (>20)", "WARN")
    
    # Perform calculations
    logger.log("Calculating daily statistics")
    daily_means = np.mean(data, axis=0)
    daily_maxes = np.max(data, axis=0)
    daily_mins = np.min(data, axis=0)
    
    # Validate results
    logger.assert_condition(len(daily_means) == data.shape[1], "Daily means length mismatch")
    logger.assert_condition(np.all(daily_mins <= daily_means), "Mins should be <= means")
    logger.assert_condition(np.all(daily_means <= daily_maxes), "Means should be <= maxes")
    
    logger.log(f"Analysis completed successfully")
    
    return {
        'daily_means': daily_means,
        'daily_maxes': daily_maxes,
        'daily_mins': daily_mins,
        'overall_mean': np.mean(data),
        'data_shape': data.shape
    }

# Test with debugging enabled
debug_logger = DebugLogger(debug_enabled=True)
result = robust_inflammation_analysis(data, debug_logger)

print(f"\nAnalysis completed. Overall mean inflammation: {result['overall_mean']:.2f}")

## Best Practices Summary

Here are the key debugging principles and practices we've covered:

In [ ]:
# Debugging best practices checklist
debugging_checklist = {
    "Before debugging": [
        "Know exactly what the code should do",
        "Have test cases with known correct answers",
        "Create simplified test cases",
        "Ensure you're testing the right code/data"
    ],
    "During debugging": [
        "Make the bug reproducible (fail every time)",
        "Add debugging prints to trace execution",
        "Change one thing at a time",
        "Test each change immediately",
        "Keep detailed notes of what you try"
    ],
    "Debugging tools": [
        "Print statements for quick insights",
        "Python debugger (pdb) for step-by-step execution",
        "Exception handling for graceful error management",
        "Visualizations to understand data patterns",
        "Assertions to validate assumptions"
    ],
    "After debugging": [
        "Re-run ALL tests to check for regressions",
        "Add tests to prevent the same bug recurring",
        "Document what was wrong and how it was fixed",
        "Clean up debugging code (or make it toggleable)"
    ]
}

print("=== DEBUGGING BEST PRACTICES CHECKLIST ===")
for category, practices in debugging_checklist.items():
    print(f"\n{category.upper()}:")
    for i, practice in enumerate(practices, 1):
        print(f"  {i}. {practice}")

## Summary

In this lesson, we learned systematic approaches to debugging:

1. **Know what code should do** - Write tests with known correct answers
2. **Make it fail consistently** - Create reproducible test cases
3. **Make it fail fast** - Localize problems to small code regions
4. **Change one thing at a time** - Systematic rather than random fixes
5. **Keep detailed records** - Track what you've tried and what worked
6. **Be humble** - Ask for help when stuck, learn from mistakes

### Key Points

- Know what code is supposed to do *before* trying to debug it
- Make it fail every time
- Make it fail fast
- Change one thing at a time, and for a reason
- Keep track of what you've done
- Be humble - ask for help when needed

Systematic debugging saves time in the long run and leads to more robust, reliable code. The investment in learning these techniques pays off quickly as your programs become more complex.